In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d

In [2]:
micro_data = pd.read_pickle('../divestdata/P3/micro/unified_trajectory')
macro_data = pd.read_pickle('../divestdata/P3/aggregate/unified_trajectory')
rep_data = pd.read_pickle('../divestdata/P3/res/unified_trajectory')
print(macro_data.index.names)
print(macro_data.index.levels)
levels=['b_d', 'phi', 'tau', 'eps']
key=[1.25, 0.5, 1.0, 0.01]
micro_data.xs(level=levels, key=key)

['b_d', 'phi', 'tau', 'eps', 'timesteps', 'observables']
[[1.0, 1.25, 1.5], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], [0.1, 0.316227766017, 1.0, 3.16227766017, 10.0, 31.6227766017, 100.0, 316.227766017, 1000.0], [0.05], [0.0, 5.05050505051, 10.101010101, 15.1515151515, 20.202020202, 25.2525252525, 30.303030303, 35.3535353535, 40.404040404, 45.4545454545, 50.5050505051, 55.5555555556, 60.6060606061, 65.6565656566, 70.7070707071, 75.7575757576, 80.8080808081, 85.8585858586, 90.9090909091, 95.9595959596, 101.01010101, 106.060606061, 111.111111111, 116.161616162, 121.212121212, 126.262626263, 131.313131313, 136.363636364, 141.414141414, 146.464646465, 151.515151515, 156.565656566, 161.616161616, 166.666666667, 171.717171717, 176.767676768, 181.818181818, 186.868686869, 191.919191919, 196.96969697, 202.02020202, 207.070707071, 212.121212121, 217.171717172, 222.222222222, 227.272727273, 232.323232323, 237.373737374, 242.424242424, 247.474747475, 252.525252525, 257.575757576, 2

key                    mean_trajectory  sem_trajectory
timesteps observables                                 
0.0       W_c                 2.577053    1.534424e-13
          W_d                 2.577053    1.546852e-13
          c                  28.381950    8.309680e-11
          g                  99.999952    7.070099e-13
          i_c                 0.130000    1.242207e-16
          k_c                 3.499994    6.543867e-13
          k_d                18.597649    4.375790e-12
          l_c                 0.200593    1.360637e-13
          l_d                 0.799407    1.360185e-13
          n_c                 0.130000    0.000000e+00
          r                   0.481973    9.772143e-14
          r_c                 0.116621    9.731647e-14
          r_d                 0.116621    3.981678e-15
          time                0.000000    0.000000e+00
          w                   2.713111    8.375286e-14
2.5       W_c                 2.556440    3.325417e-03
          W_d                 2.556225    1.964730e-03
          c                  26.917615    1.521240e-02
          g                  98.795951    1.763864e-03
          i_c                 0.132522    4.336208e-02
          k_c                 3.381602    7.271958e-02
          k_d                18.442261    7.221289e-02
          l_c                 0.196542    3.189787e-03
          l_d                 0.803458    3.189787e-03
          n_c                 0.132525    4.338690e-02
          r                   0.481170    1.896921e-03
          r_c                 0.117158    6.108556e-04
          r_d                 0.117079    4.586698e-06
          time                2.500000    0.000000e+00
          w                   2.687377    2.734980e-04
...                                ...             ...
497.5     W_c                 1.853862    2.067494e-02
          W_d                 1.311118    6.009915e-01
          c                  80.555472    1.915021e+00
          g                  43.475817    4.348473e+00
          i_c                 0.956557    5.211737e-02
          k_c                15.738426    7.173613e-01
          k_d                 0.681938    5.421541e-01
          l_c                 0.988920    1.180035e-02
          l_d                 0.011080    1.180035e-02
          n_c                 0.953901    5.468206e-02
          r                   0.010659    9.779505e-03
          r_c                 0.115386    2.667158e-03
          r_d                 0.034810    1.477413e-02
          time              497.500000    0.000000e+00
          w                   2.445622    3.903645e-02
500.0     W_c                 1.854365    2.186480e-02
          W_d                 1.293854    6.239082e-01
          c                  80.565049    1.911390e+00
          g                  43.449068    4.335330e+00
          i_c                 0.956168    5.081322e-02
          k_c                15.737781    7.355036e-01
          k_d                 0.684250    5.638030e-01
          l_c                 0.988785    1.242202e-02
          l_d                 0.011215    1.242202e-02
          n_c                 0.953475    5.387956e-02
          r                   0.010740    1.024738e-02
          r_c                 0.115388    2.739350e-03
          r_d                 0.034722    1.475674e-02
          time              500.000000    0.000000e+00
          w                   2.445749    3.955742e-02

[3015 rows x 2 columns]

In [37]:
def l_2_distance(d1, d2, key=None, levels=None, axes=None, observable=None):
    
    names = d1.index.names
    indices = {ax: d1.index.levels[names.index(ax)].values for ax in axes}
    output = np.zeros((len(indices['tau']),len(indices['phi'])))
    
    print(indices)
    
    for i, tau in enumerate(indices['tau']):
        for j, phi in enumerate(indices['phi']):
            tkey = key + [phi, tau]
    
            # get values and times from dataframes
            d1_slice = d1.xs(key=tkey, level=levels)['mean_trajectory'].unstack('observables')[observable].values.T[0]
            d1_times = d1.xs(key=tkey, level=levels)['mean_trajectory'].unstack('observables').index.values

            d2_slice = d2.xs(key=tkey, level=levels)['mean_trajectory'].unstack('observables')[observable].values.T[0]
            d2_times = d2.xs(key=tkey, level=levels)['mean_trajectory'].unstack('observables').index.values

            # interpolate to get values on equal timepoints
            d1_ipl = interp1d(d1_times, d1_slice)
            d2_ipl = interp1d(d2_times, d2_slice)

            samples = 200
            times = np.linspace(d1_times[0], d1_times[-1], samples)
            d1_data = d1_ipl(times)
            d2_data = d2_ipl(times)
            dd = d1_data - d2_data
            l2 = sum(dd*dd)
            output[i, j] = l2
            print(l2, tau, phi, i, j)
            
            return output

In [38]:
axes=['phi', 'tau']
levels=['b_d', 'eps', 'phi', 'tau']
key=[1.25, 0.05]


l_2_distance(micro_data, macro_data, key=key, levels=levels, axes=axes, observable=['g'])

{'phi': array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 'tau': array([  1.00000000e-01,   3.16227766e-01,   1.00000000e+00,
         3.16227766e+00,   1.00000000e+01,   3.16227766e+01,
         1.00000000e+02,   3.16227766e+02,   1.00000000e+03])}
40.0342507016 0.1 0.0 0 0


array([[ 40.0342507,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
          0.  